In [32]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

___
Enter Team Member Names here (*double click to edit*):

- Name 1: Jake Carlson
- Name 2: Eric Smith

___

# In Class Assignment Two
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class (or right after class). The initial portion of this notebook is given before class and the remainder is given during class. Please answer the initial questions before class, to the best of your ability. Once class has started you may rework your answers as a team for the initial part of the assignment. 

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#svm">Linear SVMs</a>
* <a href="#svm_using">Using Linear SVMs</a>
* <a href="#nonlinear">Non-linear SVMs</a>

________________________________________________________________________________________________________

<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Loading the Data
Please run the following code to read in the "olivetti faces" dataset from sklearn's data loading module. 

This will load the data into the variable `ds`. `ds` is a `bunch` object with fields like `ds.data` and `ds.target`. The field `ds.data` is a numpy matrix of the continuous features in the dataset. **The object is not a pandas dataframe. It is a numpy matrix.** Each row is a set of observed instances, each column is a different feature. It also has a field called `ds.target` that is an integer value we are trying to predict (i.e., a specific integer represents a specific person). Each entry in `ds.target` is a label for each row of the `ds.data` matrix. 

In [33]:
# fetch the images for the dataset
# this will take a long time the first run because it needs to download
# after the first time, the dataset will be save to your disk (in sklearn package somewhere) 
# if this does not run, you may need additional libraries installed on your system (install at your own risk!!)
from sklearn.datasets import fetch_lfw_people

lfw_people = fetch_lfw_people(min_faces_per_person=20, resize=None)

In [34]:
# get some of the specifics of the dataset
X = lfw_people.data
y = lfw_people.target
names = lfw_people.target_names

n_samples, n_features = X.shape
_, h, w = lfw_people.images.shape
n_classes = len(names)

print("n_samples: {}".format(n_samples))
print("n_features: {}".format(n_features))
print("n_classes: {}".format(n_classes))
print("Original Image Sizes {} by {}".format(h,w))
print (125*94) # the size of the images are the size of the feature vectors

n_samples: 3023
n_features: 11750
n_classes: 62
Original Image Sizes 125 by 94
11750


**Question 1:** For the faces dataset, describe what the data represents? That is, what is each column? What is each row? What do the unique class values represent?



*Enter you answer here (double click)*

This data represents cropped pictures of faces. Each row is a unique image and each column contains a single pixel value in that image. The class of each picture is the name of the subject.

___

<a id="svm"></a>
<a href="#top">Back to Top</a>
## Linear Support Vector Machines

**Question 2:** If we were to train a linear Support Vector Machine (SVM) upon the faces data, how many parameters would need to be optimized in the model? That is, how many coefficients would need to be calculated?




*Enter you answer here (double click)*

We would need weights foe the 11750 features, plus a bias term, times the number of classes: 728,562.

___

In [35]:
# Enter any scratchwork or calculations here
(11750+1) * 62

728562

**Question 3:** 
- **Part A:** Given the number of parameters calculated above, would you expect the model to train quickly using **batch optimization techniques**? Why or why not? 
- **Part B:** Is there a way to reduce training time?
- **Part C:** If we transformed the X data using principle components analysis (PCA) with 100 components, how many parameters would we need to find for a linear Support Vector Machine (SVM)?

*Enter you answer here (double click)*


A. It will take a long time to train the model because we have to maximize the alpha margin distance values for every combination of weights. The Dual can only be solved numerically, and it cannot be parallelized well.

B. We can use PCA to reduce the dimensionality of our data, there by reducing the number of weights that need to be solved for.

C. We would need to find 6262 weights to train our model.
___

In [36]:
# Enter any scratchwork or calculations here
n = (100+1)*62
print('Part C. With 100 features: ', n)


Part C. With 100 features:  6262


___
<a id="svm_using"></a>
<a href="#top">Back to Top</a>

# Using Linear SVMs

**Exercise 1:** Use the block of code below to check if the number of parameters you calculated is equal to the number of parameters returned by `sklearn`'s implementation of the Linear SVM. **Was your calculation correct? If different, can you think of a reason why the parameters would not match?**

In [47]:
from sklearn.svm import LinearSVC
from sklearn.decomposition import RandomizedPCA

n_components = 100
pca = RandomizedPCA(n_components=n_components)
Xpca = pca.fit_transform(X)

clf = LinearSVC()
clf.fit(Xpca,y)

/Users/JakeCarlson/anaconda/envs/MLEnv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:57: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [48]:
#===================================================================
# Enter your code below to calculate the number of parameters in the model 
print(len(clf.coef_)*len(clf.coef_[0]))
#===================================================================

6200


It looks like sklearn's implementation leaves out the bias from the coefficient matrix.

___
**Exercise 2:** Use the starter code below to calculate two quantities: 
- **Part A.:** The overall accuracy of the trained linear svm on the training set
- **Part B.:** The *mean, standard deviation, maximum, and minimum* of the **accuracy per class** on the training set

You might be interested in the following documentation of the confusion matrix calculated by `scikit-learn`:
- http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

And an example matrix returned by the confusion matrix function:
<img src="http://scikit-learn.org/stable/_images/sphx_glr_plot_confusion_matrix_001.png",width=400,height=400>

In [49]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np

yhat = clf.predict(Xpca)

#===================================================
# Enter your code below

print('Overall Accuracy is ',clf.score(Xpca, y))

cm = confusion_matrix(y, yhat)
diag = cm.diagonal()

class_acc = [0 for i in range(cm.shape[0])]
for i in range(cm.shape[0]):
    class_acc[i] = diag[i] / sum(cm[i])


print('The class accuracy is ',np.mean(class_acc), '+-', np.std(class_acc),end=' ')
print('(min,max) (',min(class_acc), max(class_acc),')')

#===================================================

Overall Accuracy is  0.873966258683
The class accuracy is  0.893211968412 +- 0.116430592413 (min,max) ( 0.55 1.0 )


___
<a id="nonlinear"></a>
<a href="#top">Back to Top</a>

# Non-linear SVMs
Now let's explore the use of non-linear svms. More explicitly, using different kernels. Take a look at the example training and testing code below for the non-linear SVM. All parameters are left as default, except we change the kernel to be `rbf`. Run the block of code below.



In [50]:
from sklearn.svm import SVC

clf = SVC(kernel='rbf')
clf.fit(Xpca,y)
yhat = clf.predict(Xpca)
print('Overall Accuracy is ',accuracy_score(y,yhat))

Overall Accuracy is  0.935494541846


___
**Exercise 3:** Use the starter code from above to calculate the accuracy for three different non-linear SVM kernels. That is, repeat the code above for different `kernel` parameters. **Only use non-linear kernels.  Which kernel is most accurate with the default parameters?**

You might be interested in the documentation of the scikit-learn SVM implementation, available here:
- http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [52]:
#===================================================
# Enter your code below
clf = SVC(kernel='poly')
clf.fit(Xpca,y)
yhat = clf.predict(Xpca)
print('Overall Accuracy poly is ',accuracy_score(y,yhat))

clf = SVC(kernel='sigmoid')
clf.fit(Xpca,y)
yhat = clf.predict(Xpca)
print('Overall Accuracy sigmoid is ',accuracy_score(y,yhat))

clf = SVC(kernel='rbf')
clf.fit(Xpca,y)
yhat = clf.predict(Xpca)
print('Overall Accuracy rbf is ',accuracy_score(y,yhat))
 

#===================================================

Overall Accuracy poly is  0.974197816738
Overall Accuracy sigmoid is  0.213364207741
Overall Accuracy rbf is  0.935494541846


*Enter you answer here (double click)*

The most accurate kernel is poly.


___
**Exercise 4:** Choose the **most accurate kernel** and manipulate the settings for `gamma` to make the classification more accurate. 
- **Part A:** How accurate can you make it? 
- **Part B:** Would you expect the results to generalize well? Why or why not?

In [56]:
#===================================================
# Enter your code below

kern = 'poly'
g = 1/(2*10)

yhat = SVC(kernel=kern, gamma=g).fit(Xpca,y).predict(Xpca)
print('Overall Accuracy is ', accuracy_score(y,yhat)) 

#===================================================

Overall Accuracy is  1.0


*Enter you answer here (double click)*

A. The highest accuracy we could achieve was 100%.

B. We would not expect this to generalize because this gamma value resulted in 100% accuracy for this training set, which likely means the model is over-fitted for the training set. If we gave the model a photo it hasn't seen, but a subject that is in the model, it will likely classify it correctly. However, we would expect a lower classification accuracy for a test data set that contains subjects that haven't been seen before.



___
**Final Question:** Using the most accurate non-linear SVM you found in the previous question, how many parameter coefficients does the trained model contain?

*Enter you answer here (double click)*

The poly kernel calculated the support vectors for our data, which are the non-zero. The parameters for the decision funciton are contained in `dual_coef_`. The number of coefficients in the decision matrix are 179,401, however, we will only need to consider values in the coefficient matrix that are non-zero, because classification will not consider these values. Therefore, we only need 61197 parameter coefficients.


In [69]:
#===================================================
# Enter any scratchwork calculations you need below
poly_fit = SVC(kernel=kern, gamma=g).fit(Xpca,y)
print(poly_fit.dual_coef_.shape[0] * poly_fit.dual_coef_.shape[1])
print("--------------")

n_non_zero = 0
for i in range(poly_fit.dual_coef_.shape[0]):
    for j in range(len(poly_fit.dual_coef_[i])):
        if poly_fit.dual_coef_[i][j] > 0:
            n_non_zero += 1
print(n_non_zero)

print("--------------")
print(poly_fit.dual_coef_.shape)


179401
--------------
61197
--------------
(61, 2941)


________________________________________________________________________________________________________

That's all! Please **save (make sure you saved!!!) and upload your rendered notebook** and please include **team member names** in the notebook submission.